# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive import normal, uniform, choice


from azureml.core.experiment import Experiment
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'heart_disease'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [4]:
cpu_cluster_name = "capstone-comp"
# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Hyperdrive Configuration and Run Details

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5) 

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
        "--C": uniform(0.05, 0.1),
        "--max_iter": choice(16, 32, 64, 128)})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='./', 
                    entry_script='train.py', 
                    compute_target=cpu_cluster)

hyperdrive_run_config = HyperDriveConfig (estimator = estimator,
                             hyperparameter_sampling = param_sampling,
                             policy = early_termination_policy,
                             primary_metric_name = "Accuracy",
                             primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs = 25,
                             max_concurrent_runs = 3
                             )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [8]:
#TODO: Submit your experiment
experiment = Experiment(ws,'heart_disease')
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('best run ID: ', best_run.id)
print('\n Accuracy: ',best_run_metrics['Accuracy'])

best run ID:  HD_02bdcb98-cc0a-4a40-ab85-c5b207d9a275_0

 Accuracy:  0.8571428571428571


In [10]:
#TODO: Save the best model
model = best_run.register_model(model_name='heart_disease_check', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service